In [ ]:
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
import sys

ROOT_DIR = os.path.abspath("../src")
sys.path.append(ROOT_DIR)
import utils

In [ ]:
# dataset_dir = '/media/aaron/DATA/ubuntu/fpha-dataset/'
dataset_dir = '/media/aaron/DATADRIVE1/First_Person_Action_Benchmark'
image_dir = os.path.join(dataset_dir, 'Video_files')
annot_dir = os.path.join(dataset_dir, 'Hand_pose_annotation_v1')

In [ ]:
subject_list = [x for x in os.listdir(image_dir) 
              if os.path.isdir(os.path.join(image_dir, x))]
print(subject_list)

In [ ]:
action_list = [x for x in os.listdir(os.path.join(image_dir, subject_list[0])) 
              if os.path.isdir(os.path.join(os.path.join(image_dir, subject_list[0]), x))]
print(action_list)

In [ ]:
img_path = os.path.join(image_dir, subject_list[0], action_list[0]) 

video_id_list = [x for x in os.listdir(img_path) 
              if os.path.isdir(os.path.join(img_path, x))]
print(video_id_list)

In [ ]:
img_path_colour = os.path.join(img_path, video_id_list[0], 'color')
img_path_depth = os.path.join(img_path, video_id_list[0], 'depth')

img_colour_list = [x for x in os.listdir(img_path_colour) 
              if os.path.join(img_path_colour, x)]
img_depth_list = [x for x in os.listdir(img_path_depth) 
              if os.path.join(img_path_depth, x)]

In [ ]:
sample = {
    'subject': subject_list[0],
    'action_name': action_list[0],
    'seq_idx': video_id_list[0],
    'frame_idx': 0,
    'object': None
}


img_color_path = os.path.join(image_dir, 
                   sample['subject'], 
                   sample['action_name'],
                   sample['seq_idx'],
                   'color',
                   'color_%04d.jpeg' %sample['frame_idx'])

img_depth_path = os.path.join(image_dir, 
                   sample['subject'], 
                   sample['action_name'],
                   sample['seq_idx'],
                   'depth',
                   'depth_%04d.png' %sample['frame_idx'])

img_color = cv2.imread(img_color_path)[:,:,::-1]
img_depth = cv2.imread(img_depth_path)[:,:,0]

# fig, ax = plt.subplots(1, 2, figsize=(18, 16))
# ax[0].imshow(img_color[:, :, ::-1])
# ax[0].set_title('color')
# ax[1].imshow(img_depth[:,:,0])
# ax[1].set_title('depth')

In [ ]:
reorder_idx = np.array([
    0, 1, 6, 7, 8, 2, 9, 10, 11, 3, 12, 13, 14, 4, 15, 16, 17, 5, 18, 19,
    20
])
skel = utils.get_skeleton(sample, annot_dir)[reorder_idx]
print('skel=', skel)

In [ ]:
skeleton_path = os.path.join(annot_dir, sample['subject'],
                             sample['action_name'], sample['seq_idx'],
                             'skeleton.txt')
skeleton_vals = np.loadtxt(skeleton_path)
skeleton = skeleton_vals[:, 1:].reshape(skeleton_vals.shape[0], 21,
                                        -1)
print('min depth=', np.min(skeleton[:,:,2]))
print('max depth=',np.max(skeleton[:,:,2]))

In [ ]:
print('color.shape=', img_color.shape)
print('depth.shape=', img_depth.shape)
fig, ax = plt.subplots(1, 2, figsize=(18, 16))
ax[0].imshow(img_color)
ax[0].set_title('color')
ax[1].imshow(img_depth, cmap='gray')
ax[1].set_title('depth')

#xyz to uvd

cam_extr = np.array(
    [[0.999988496304, -0.00468848412856, 0.000982563360594,
      25.7], [0.00469115935266, 0.999985218048, -0.00273845880292, 1.22],
     [-0.000969709653873, 0.00274303671904, 0.99999576807,
      3.902], [0, 0, 0, 1]])
cam_intr_color = np.array([[1395.749023, 0, 935.732544],
                     [0, 1395.749268, 540.681030], [0, 0, 1]])
cam_intr_depth = np.array([[475.065948, 0,  315.944855],
                     [0, 475.065857, 245.287079], [0, 0, 1]])

skel_hom = np.concatenate([skel, np.ones([skel.shape[0], 1])], 1)
# print('skel_hom=', skel_hom)
skel_camcoords = cam_extr.dot(
    skel_hom.transpose()).transpose()[:, :3].astype(np.float32)
# print('skel_camcoords=', skel_camcoords)

skel_hom2d_color = np.array(cam_intr_color).dot(skel_camcoords.transpose()).transpose()
skel_proj_color_3d = (skel_hom2d_color / skel_hom2d_color[:, 2:])
skel_proj_color = skel_proj_color_3d[:, :2]

skel_hom2d_depth = np.array(cam_intr_depth).dot(skel.transpose()).transpose()
skel_proj_depth_3d = (skel_hom2d_depth / skel_hom2d_depth[:, 2:])
skel_proj_depth = skel_proj_depth_3d[:, :2]

utils.visualize_joints_2d(ax[0], skel_proj_color, joint_idxs=False)
utils.visualize_joints_2d(ax[1], skel_proj_depth, joint_idxs=False)

In [ ]:
skel_uvd_color = skel_hom2d_color
skel_uvd_color[:, :2] =  (skel_hom2d_color[:, :2]/skel_hom2d_color[:, 2:])

#uvd to xyz
skel_uvd_color[:, :2] = skel_uvd_color[:, :2]*skel_uvd_color[:, 2:]

inv_cam_intr_color = np.linalg.inv(cam_intr_color)
skel_camcoords_2 = np.array(inv_cam_intr_color).dot(skel_uvd_color.transpose()).transpose()
skel_camcoords_2 = np.concatenate([skel_camcoords_2, np.ones([skel_camcoords.shape[0], 1])], 1)
inv_cam_extr_color = np.linalg.inv(cam_extr)
skel_xyz_color = np.array(inv_cam_extr_color).dot(skel_camcoords_2.transpose()).transpose()

# print('reconstructed xyz=', skel_xyz_color[:, :3])
# print('original xyz=', skel)

In [ ]:
import imgaug as ia
from imgaug import augmenters as iaa

#color
seq = iaa.Sequential([
    iaa.size.Resize({"height": 416, "width": 416}),
    iaa.ChangeColorspace(from_colorspace="RGB", to_colorspace="HSV"),
    iaa.WithChannels(0, iaa.Add((0, 30))), #hue
    iaa.WithChannels(1, iaa.Add((0, 30))), #saturation
    iaa.WithChannels(2, iaa.Add((0, 60))), #exposure
    iaa.ChangeColorspace(from_colorspace="HSV", to_colorspace="RGB"),
    iaa.Affine(translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}),
])

seq_det = seq.to_deterministic()

skel_color_kps = []
for kps in skel_proj_color:
    skel_color_kps.append(ia.Keypoint(x=kps[0],y=kps[1]))
skel_color_kpsoi = ia.KeypointsOnImage(skel_color_kps, shape=img_color.shape)

images_aug_color = seq_det.augment_images([img_color])[0]
skel_aug_color = seq_det.augment_keypoints([skel_color_kpsoi])[0]

skel_proj_color_aug = []
for kps in skel_aug_color.keypoints:
    skel_proj_color_aug.append([kps.x_int, kps.y_int])
skel_proj_color_aug = np.asarray(skel_proj_color_aug)

#depth
seq = iaa.Sequential([
    iaa.size.Resize({"height": 416, "width": 416}),
    iaa.Affine(translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}),
])

seq_det = seq.to_deterministic()

skel_depth_kps = []
for kps in skel_proj_depth:
    skel_depth_kps.append(ia.Keypoint(x=kps[0],y=kps[1]))
skel_depth_kpsoi = ia.KeypointsOnImage(skel_depth_kps, shape=img_depth.shape)
    
images_aug_depth = seq_det.augment_images([img_depth])[0]
skel_aug_depth = seq_det.augment_keypoints([skel_depth_kpsoi])[0]

skel_proj_depth_aug = []
for kps in skel_aug_depth.keypoints:
    skel_proj_depth_aug.append([kps.x_int, kps.y_int])
skel_proj_depth_aug = np.asarray(skel_proj_depth_aug)

fig, ax = plt.subplots(1, 2, figsize=(18, 16))
ax[0].imshow(images_aug_color)
ax[0].set_title('color')
ax[1].imshow(images_aug_depth)
ax[1].set_title('depth')


utils.visualize_joints_2d(ax[0], skel_proj_color_aug, joint_idxs=False)
utils.visualize_joints_2d(ax[1], skel_proj_depth_aug, joint_idxs=False)

In [ ]:
#get split data

train_pairs = []
test_pairs = []
with open(os.path.join(dataset_dir, 'data_split_action_recognition.txt')) as f:
    cur_split = 'Training'
    lines = f.readlines()
    for l in lines:
        words = l.split()
        if(words[0] == 'Training' or words[0] == 'Test'):
            cur_split = words[0]
        else:
            path = l.split()[0]
            full_path = os.path.join(image_dir, path, 'color')
            len_frame_idx = len([x for x in os.listdir(full_path)
                                if os.path.join(full_path, x)])
            skeleton_path = os.path.join(annot_dir, path, 'skeleton.txt')
            skeleton_vals = np.loadtxt(skeleton_path)
            for i in range(len_frame_idx):
                img_path = os.path.join(image_dir, path, 'color', 'color_%04d.jpeg' %i)
                skel_xyz = skeleton_vals[:, 1:].reshape(skeleton_vals.shape[0], -1)[i]
                data_pair = (img_path, skel_xyz)
                if cur_split == 'Training':
                    train_pairs.append(data_pair)
                else:
                    test_pairs.append(data_pair)
                    
=